# Cell 0: imports & config

In [1]:
import os, json, math, hashlib, shutil, time
from pathlib import Path
from typing import List, Dict, Any, Tuple

from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_groq import ChatGroq

from IPython.display import display, HTML

# --- Config ---
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")  # expects to be set in your .env

CACHE_DIR = Path(".yt_rag_cache")
CACHE_DIR.mkdir(exist_ok=True)

TARGET_WINDOW_SECONDS = 75     # ~60–90s is a good default for long videos
MMR_K = 6                      # final doc count sent to the LLM
MMR_FETCH_K = 24               # pool for diversity
K_CTX = 1800                   # safety cap on context size
MODEL_NAME = "llama-3.1-8b-instant"  # flip to 'llama-3.1-70b-versatile' for tougher Qs

print("✅ Setup complete. Cache dir:", CACHE_DIR.resolve())


✅ Setup complete. Cache dir: C:\Users\Lenovo\Projects\QnA on YouTube Video - RAG\notebooks\.yt_rag_cache


# Cell 1: utilities


In [2]:
def _ts(sec: float) -> str:
    sec = int(sec)
    h = sec // 3600
    m = (sec % 3600) // 60
    s = sec % 60
    return f"{h:d}:{m:02d}:{s:02d}" if h else f"{m:d}:{s:02d}"

def _vid_dir(video_id: str) -> Path:
    return CACHE_DIR / video_id

def _hash_text(text: str) -> str:
    import hashlib
    return hashlib.md5(text.encode("utf-8")).hexdigest()

def show_citations(video_id: str, docs: List[Document]):
    rows = []
    for d in docs:
        s = int(d.metadata["start"])
        w = f'{_ts(d.metadata["start"])}–{_ts(d.metadata["end"])}'
        url = f"https://www.youtube.com/watch?v={video_id}&t={s}s"
        rows.append(f"<tr><td>{w}</td><td><a href='{url}' target='_blank'>{url}</a></td></tr>")
    html = "<table><tr><th>Window</th><th>Jump Link</th></tr>" + "".join(rows) + "</table>"
    display(HTML(html))


# Cell 2: transcript + chunking


In [3]:
def fetch_transcript(video_id: str, languages: List[str] = ["en"]) -> List[Dict[str, Any]]:
    try:
        t0 = time.time()
        api = YouTubeTranscriptApi()
        fetched = api.fetch(video_id, languages=languages)
        data = fetched.to_raw_data()
        print(f"✅ Transcript fetched: {len(data)} segments in {time.time()-t0:.2f}s")
        return data
    except TranscriptsDisabled:
        raise RuntimeError("No captions available for this video.")
    except Exception as e:
        raise RuntimeError(f"Failed to fetch transcript: {e}")

def group_segments(segments: List[Dict[str, Any]], target_window_s: int = TARGET_WINDOW_SECONDS) -> List[Dict[str, Any]]:
    out = []
    cur, start, end = [], None, None
    for row in segments:
        t0 = row["start"]
        t1 = row["start"] + row.get("duration", 0)
        if start is None:
            start = t0
        end = t1
        cur.append(row["text"].strip())
        if (end - start) >= target_window_s:
            out.append({"start": start, "end": end, "text": " ".join(cur).strip()})
            cur, start, end = [], None, None
    if cur:
        out.append({"start": start, "end": end, "text": " ".join(cur).strip()})
    print(f"🧩 Windows created: {len(out)} (≈{target_window_s}s each)")
    return out

def make_docs(video_id: str, windows: List[Dict[str, Any]]) -> List[Document]:
    docs = []
    for i, w in enumerate(windows):
        meta = {"video_id": video_id, "start": w["start"], "end": w["end"], "window_id": i}
        docs.append(Document(page_content=w["text"], metadata=meta))
    return docs

# Demo on a video id (you can change it any time)
VIDEO_ID = "3qHkcs3kG44"
segments = fetch_transcript(VIDEO_ID)
windows = group_segments(segments)

print("👀 Preview first 2 windows:")
for w in windows[:2]:
    print(f"- [{_ts(w['start'])}–{_ts(w['end'])}] {w['text'][:160]}{'...' if len(w['text'])>160 else ''}")

docs = make_docs(VIDEO_ID, windows)
print(f"📄 Document objects: {len(docs)}")


✅ Transcript fetched: 4076 segments in 2.54s
🧩 Windows created: 108 (≈75s each)
👀 Preview first 2 windows:
- [0:00–1:16] two one boom all right we're live thank you very much for doing this man i really appreciate it i've been absorbing your information and listening to you talk f...
- [1:13–2:30] multivariate but we get summarized in pithy ways in our lives and at some deep level we know that's not true right every human basically is capable of every exp...
📄 Document objects: 108


# Cell 3: index build/load with cache


In [4]:
def build_or_load_index(video_id: str, force_rebuild: bool = False) -> Tuple[FAISS, FastEmbedEmbeddings]:
    vdir = _vid_dir(video_id)
    idx_dir = vdir / "faiss"
    meta_fp = vdir / "meta.json"

    if force_rebuild and vdir.exists():
        shutil.rmtree(vdir)
    vdir.mkdir(exist_ok=True)

    if idx_dir.exists() and meta_fp.exists():
        t0 = time.time()
        embeddings = FastEmbedEmbeddings()
        vs = FAISS.load_local(str(idx_dir), embeddings, allow_dangerous_deserialization=True)
        print(f"⚡ Loaded cached index in {time.time()-t0:.2f}s ({idx_dir})")
        return vs, embeddings

    # build fresh
    print("🔨 Building new index...")
    t0 = time.time()
    segs = fetch_transcript(video_id)
    wins = group_segments(segs)
    _docs = make_docs(video_id, wins)
    embeddings = FastEmbedEmbeddings()
    vs = FAISS.from_documents(_docs, embeddings)
    FAISS.save_local(vs, str(idx_dir))
    with open(meta_fp, "w", encoding="utf-8") as f:
        json.dump({"video_id": video_id, "num_windows": len(wins), "chunk_sec": TARGET_WINDOW_SECONDS}, f)
    print(f"✅ Index built & saved in {time.time()-t0:.2f}s → {idx_dir}")
    return vs, embeddings

vs, emb = build_or_load_index(VIDEO_ID, force_rebuild=False)


🔨 Building new index...
✅ Transcript fetched: 4076 segments in 2.40s
🧩 Windows created: 108 (≈75s each)
✅ Index built & saved in 51.57s → .yt_rag_cache\3qHkcs3kG44\faiss


# Cell 4: retrieval helpers with visible outputs


In [5]:
def simple_token_overlap_score(text: str, query: str) -> int:
    def toks(s: str) -> set:
        return {t for t in s.lower().split() if len(t) > 3}
    return len(toks(text) & toks(query))

def hybrid_candidates(vs: FAISS, query: str, k_fetch: int = MMR_FETCH_K) -> List[Document]:
    pool = vs.similarity_search(query, k=k_fetch)
    pool.sort(key=lambda d: simple_token_overlap_score(d.page_content, query), reverse=True)
    return pool

def _mmr(vs: FAISS, query: str, k: int = MMR_K, fetch_k: int = MMR_FETCH_K) -> List[Document]:
    return vs.max_marginal_relevance_search(query, k=k, fetch_k=fetch_k)

def retrieve_docs(vs: FAISS, video_id: str, query: str) -> List[Document]:
    t0 = time.time()
    pool = hybrid_candidates(vs, query, k_fetch=MMR_FETCH_K)
    tmp_vs = FAISS.from_documents(pool, vs.embedding_function)  # ephemeral small VS
    top = _mmr(tmp_vs, query, k=MMR_K, fetch_k=min(len(pool), MMR_FETCH_K))
    print(f"🔎 Retrieval done in {time.time()-t0:.2f}s → {len(top)} docs")
    for d in top:
        print(f"  · [{_ts(d.metadata['start'])}–{_ts(d.metadata['end'])}] {d.page_content[:80]}...")
    print("\n🔗 Quick jump links:")
    show_citations(video_id, top)
    return top

# Try a sample retrieval (you can change this question)
sample_q = "Is nuclear fusion discussed? What do they say about it?"
top_docs = retrieve_docs(vs, VIDEO_ID, sample_q)


🔎 Retrieval done in 10.94s → 6 docs
  · [1:44:38–1:45:54] said no more innovation in airplanes right so we need a way to iterate on nuclea...
  · [2:11:41–2:11:57] [Applause] [Music] [Music] i...
  · [1:36:04–1:37:19] and that gave me some level of peace so now i have to keep asking that question ...
  · [1:08:55–1:10:10] what happened with the university is very interesting universities first when yo...
  · [1:12:37–1:13:54] bring it up everyone's got their minds made up already well it's what's uncomfor...
  · [29:34–30:50] even the printing pros absolutely and what it does is it frees people up for new...

🔗 Quick jump links:


Window,Jump Link
1:44:38–1:45:54,https://www.youtube.com/watch?v=3qHkcs3kG44&t=6278s
2:11:41–2:11:57,https://www.youtube.com/watch?v=3qHkcs3kG44&t=7901s
1:36:04–1:37:19,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5764s
1:08:55–1:10:10,https://www.youtube.com/watch?v=3qHkcs3kG44&t=4135s
1:12:37–1:13:54,https://www.youtube.com/watch?v=3qHkcs3kG44&t=4357s
29:34–30:50,https://www.youtube.com/watch?v=3qHkcs3kG44&t=1774s


# Cell 5: prompt + LLM chain


In [6]:
PROMPT = PromptTemplate(
    input_variables=["context", "question", "video_id"],
    template=(
        "Answer using ONLY the context. Include timestamped citations like [mm:ss] "
        "where relevant. If not answerable, say so.\n\n"
        "Video ID: {video_id}\n"
        "Context:\n{context}\n\n"
        "Question: {question}\n"
    ),
)
llm = ChatGroq(model=MODEL_NAME, temperature=0.2)
parser = StrOutputParser()

def _format_docs(docs: List[Document], video_id: str, max_chars: int = K_CTX) -> str:
    parts, used = [], 0
    for d in docs:
        s, e = d.metadata.get("start", 0), d.metadata.get("end", 0)
        tag = f"[{_ts(s)}–{_ts(e)}]"
        text = d.page_content.strip()
        chunk = f"{tag} {text}"
        if used + len(chunk) > max_chars and parts:
            break
        parts.append(chunk)
        used += len(chunk)
    print(f"🧵 Context length: {used} chars across {len(parts)} windows")
    return "\n\n".join(parts)


# Cell 6: single question


In [7]:
def answer_one(video_id: str, question: str) -> Dict[str, Any]:
    vs, _ = build_or_load_index(video_id, force_rebuild=False)
    top_docs = retrieve_docs(vs, video_id, question)
    context = _format_docs(top_docs, video_id)
    chain = (RunnableParallel({
                "context": RunnableLambda(lambda _: context),
                "question": RunnablePassthrough(),
                "video_id": RunnableLambda(lambda _: video_id),
            }) | PROMPT | llm | parser)
    t0 = time.time()
    text = chain.invoke(question).strip()
    print(f"🧠 LLM answered in {time.time()-t0:.2f}s")
    print("\n=== Answer ===\n")
    print(text)
    print("\n=== Citations ===")
    show_citations(video_id, top_docs)
    # structured return if you want to consume programmatically
    cits = []
    for d in top_docs:
        s = int(d.metadata["start"])
        cits.append({
            "window": f'{_ts(d.metadata["start"])}–{_ts(d.metadata["end"])}',
            "url": f"https://www.youtube.com/watch?v={video_id}&t={s}s"
        })
    return {"answer": text, "citations": cits}

# Try it:
_ = answer_one(VIDEO_ID, "Is the topic of nuclear fusion discussed? Summarize briefly.")


⚡ Loaded cached index in 2.22s (.yt_rag_cache\3qHkcs3kG44\faiss)
🔎 Retrieval done in 10.47s → 6 docs
  · [1:43:25–1:44:40] we need one sun one solar system has got all the power we will need for a long l...
  · [2:11:41–2:11:57] [Applause] [Music] [Music] i...
  · [1:12:37–1:13:54] bring it up everyone's got their minds made up already well it's what's uncomfor...
  · [1:08:55–1:10:10] what happened with the university is very interesting universities first when yo...
  · [1:38:31–1:39:47] bacteria the parasites right the symbiotic relationship we have to our environme...
  · [1:33:36–1:34:53] foundation of understanding a steel frame of understanding than it is to just ha...

🔗 Quick jump links:


Window,Jump Link
1:43:25–1:44:40,https://www.youtube.com/watch?v=3qHkcs3kG44&t=6205s
2:11:41–2:11:57,https://www.youtube.com/watch?v=3qHkcs3kG44&t=7901s
1:12:37–1:13:54,https://www.youtube.com/watch?v=3qHkcs3kG44&t=4357s
1:08:55–1:10:10,https://www.youtube.com/watch?v=3qHkcs3kG44&t=4135s
1:38:31–1:39:47,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5911s
1:33:36–1:34:53,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5616s


🧵 Context length: 1370 chars across 2 windows
🧠 LLM answered in 0.64s

=== Answer ===

Yes, the topic of nuclear fusion is discussed. The speaker mentions that nuclear fusion can provide the power needed for a long time and that we are not far from making those technologies work.

=== Citations ===


Window,Jump Link
1:43:25–1:44:40,https://www.youtube.com/watch?v=3qHkcs3kG44&t=6205s
2:11:41–2:11:57,https://www.youtube.com/watch?v=3qHkcs3kG44&t=7901s
1:12:37–1:13:54,https://www.youtube.com/watch?v=3qHkcs3kG44&t=4357s
1:08:55–1:10:10,https://www.youtube.com/watch?v=3qHkcs3kG44&t=4135s
1:38:31–1:39:47,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5911s
1:33:36–1:34:53,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5616s


# Cell 7: batch questions


In [8]:
def answer_batch(video_id: str, questions: List[str]) -> List[Dict[str, Any]]:
    vs, _ = build_or_load_index(video_id, force_rebuild=False)
    results = []
    for i, q in enumerate(questions, 1):
        print(f"\n——— Q{i}/{len(questions)} ———")
        top_docs = retrieve_docs(vs, video_id, q)
        context = _format_docs(top_docs, video_id)
        chain = (RunnableParallel({
                    "context": RunnableLambda(lambda _: context),
                    "question": RunnablePassthrough(),
                    "video_id": RunnableLambda(lambda _: video_id),
                }) | PROMPT | llm | parser)
        t0 = time.time()
        text = chain.invoke(q).strip()
        print(f"🧠 LLM answered in {time.time()-t0:.2f}s")
        print("Answer:", text[:300] + ("..." if len(text) > 300 else ""))
        cits = []
        for d in top_docs:
            s = int(d.metadata["start"])
            cits.append({
                "window": f'{_ts(d.metadata["start"])}–{_ts(d.metadata["end"])}',
                "url": f"https://www.youtube.com/watch?v={video_id}&t={s}s"
            })
        results.append({"question": q, "answer": text, "citations": cits})
    print("\n✅ Batch complete.")
    return results

# Example batch run (edit these):
batch_qs = [
    "List three key ideas mentioned in the introduction.",
    "What is the main argument made around the halfway point?",
    "Give a concise definition of any technical term the speaker explains.",
]
batch_results = answer_batch(VIDEO_ID, batch_qs)


⚡ Loaded cached index in 2.18s (.yt_rag_cache\3qHkcs3kG44\faiss)

——— Q1/3 ———
🔎 Retrieval done in 11.37s → 6 docs
  · [12:19–13:34] kind of person who can create wealth create business make money and my theory be...
  · [2:11:41–2:11:57] [Applause] [Music] [Music] i...
  · [35:42–36:59] works right we don't know we literally have no idea so most of the ai approaches...
  · [1:33:36–1:34:53] foundation of understanding a steel frame of understanding than it is to just ha...
  · [1:07:41–1:08:56] slouch is left right leviathan is the government why does it slouch left and i t...
  · [1:29:53–1:31:10] turn it into an incredible tourist park and put your money where your mouth is s...

🔗 Quick jump links:


Window,Jump Link
12:19–13:34,https://www.youtube.com/watch?v=3qHkcs3kG44&t=739s
2:11:41–2:11:57,https://www.youtube.com/watch?v=3qHkcs3kG44&t=7901s
35:42–36:59,https://www.youtube.com/watch?v=3qHkcs3kG44&t=2142s
1:33:36–1:34:53,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5616s
1:07:41–1:08:56,https://www.youtube.com/watch?v=3qHkcs3kG44&t=4061s
1:29:53–1:31:10,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5393s


🧵 Context length: 1488 chars across 2 windows
🧠 LLM answered in 0.60s
Answer: Three key ideas mentioned in the introduction are:

1. The three basic things everybody wants: to be wealthy, happy, and fit.
2. The importance of internal calmness and a loving household, although not as central as the initial three.
3. The idea that these qualities can be taught and learned, rathe...

——— Q2/3 ———
🔎 Retrieval done in 10.44s → 6 docs
  · [30:49–32:04] with ubi there's a couple of problems with ubi one is you're creating a straight...
  · [1:36:04–1:37:19] and that gave me some level of peace so now i have to keep asking that question ...
  · [1:01:31–1:02:48] taken on liability so they're sliding down the slippery slope into ruin sloping ...
  · [34:28–35:44] assumption that all of the computation is going at the cellular level at the neu...
  · [1:15:04–1:16:21] why i stay out of politics largely do they drag you in though sometimes they alw...
  · [1:44:38–1:45:54] said no more innovation 

Window,Jump Link
30:49–32:04,https://www.youtube.com/watch?v=3qHkcs3kG44&t=1849s
1:36:04–1:37:19,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5764s
1:01:31–1:02:48,https://www.youtube.com/watch?v=3qHkcs3kG44&t=3691s
34:28–35:44,https://www.youtube.com/watch?v=3qHkcs3kG44&t=2068s
1:15:04–1:16:21,https://www.youtube.com/watch?v=3qHkcs3kG44&t=4504s
1:44:38–1:45:54,https://www.youtube.com/watch?v=3qHkcs3kG44&t=6278s


🧵 Context length: 1380 chars across 1 windows
🧠 LLM answered in 0.60s
Answer: The main argument made around the halfway point is that Universal Basic Income (UBI) can lead to a slippery slope towards socialism, where the majority of people vote themselves money, potentially bankrupting the country, and also that it can lower the status and meaning of individuals who receive i...

——— Q3/3 ———
🔎 Retrieval done in 9.83s → 6 docs
  · [1:33:36–1:34:53] foundation of understanding a steel frame of understanding than it is to just ha...
  · [25:52–27:08] time externally i'll bring that in-house if it's easy to do externally and it's ...
  · [1:37:18–1:38:34] are all axioms these are all just stopping points saying simulation we're in a s...
  · [2:03:04–2:04:20] someone who is smart or someone who appears smart they say smart things oh god i...
  · [13:32–14:47] storm so there's a tweet storm with like 36 38 tweets got very famous got transl...
  · [1:44:38–1:45:54] said no more innovation i

Window,Jump Link
1:33:36–1:34:53,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5616s
25:52–27:08,https://www.youtube.com/watch?v=3qHkcs3kG44&t=1552s
1:37:18–1:38:34,https://www.youtube.com/watch?v=3qHkcs3kG44&t=5838s
2:03:04–2:04:20,https://www.youtube.com/watch?v=3qHkcs3kG44&t=7384s
13:32–14:47,https://www.youtube.com/watch?v=3qHkcs3kG44&t=812s
1:44:38–1:45:54,https://www.youtube.com/watch?v=3qHkcs3kG44&t=6278s


🧵 Context length: 1391 chars across 1 windows
🧠 LLM answered in 0.43s
Answer: The speaker explains the following technical term:

- **Scaffolding**: This refers to memorizing advanced concepts without truly understanding the underlying basics. It is a superficial approach to learning, where one may appear to be knowledgeable but lacks a deep understanding of the subject matte...

✅ Batch complete.


# Cell 8: optional tweaks (set and re-run cells that use them)


In [ ]:
# 1) Stronger model for tricky queries:
# MODEL_NAME = "llama-3.1-70b-versatile"

# 2) Rebuild the index if you change chunking settings:
# vs, emb = build_or_load_index(VIDEO_ID, force_rebuild=True)

# 3) Adjust retrieval selectivity:
# MMR_K = 6
# MMR_FETCH_K = 24
# K_CTX = 1800
print("ℹ️ Use this cell to toggle settings, then re-run Cells 3–7.")
